<h1>Pre-processing for X SNP Data</h1>

The X chromosome has often been excluded from GWAS, which means that in many older data sets it was not analyzed before publication. As a result, X chromosome data sometimes requires additional preprocessing before it can be successfully be run through plink or XWAS.

When working with GWAS data, you will first need to know the following for each data set:
<pre>
    <b>File stem:</b> Prefix of the .bed/.bim/.fam or .ped/.map files
    <b>Array:</b> Platform and version
    <b>Genome build:</b> e.g. b36 (hg18)
    <b>SNP IDs:</b> e.g. rsID or Affy SNP ID
    <b>Allele coding:</b> e.g. A/T/C/G or A/B
    <b>X chromosome coding:</b> e.g. chrX or chr23
</pre>

Most of this information can be found in the array annotation file,
e.g. GenomeWideSNP_6.na35.annot.csv, or HumanOmni1-Quad_v1-0_B.csv
(note: the format of this file varies depending on the platform).

Once you have this information, you can proceed with pre-processing the data...

<h2>*** (Steps are shown in suggested order) ***</h2>

<h2>Identifying problems with your data</h2>

I recommend running two scripts to get a sense of what pre-QC steps you will need to perform
on each data set.

First, run fam_summary.py to check the number of cases and controls of each sex that plink is reading from the .fam file. These should match the numbers listed in the Study Report PDF included in the dbGaP download package.

Second, run Kaixiong Ye's script to check that alleles match between imputation reference files and plink files. This is a good general QC check - any of the problems described below can cause low allelic concordance scores. If you've fixed all of these problems, the scores should all be >95%.

In [ ]:
## Check the number of cases and controls of each sex
python /custom_scripts/fam_summary.py data.fam

## Check allele concordance of SNPs in .bim file with imputation ref file of the correct build
perl /custom_scripts/check_genome_build_and_strange_alignment.pl data.bim /reference_files/chrX.hg19.legend data_check_hg19_X.txt

<h2>Merge samples from the same study</h2>

You may recieve a data set in parts, such as different consent groups.

It would be appropriate to merge samples if:
<pre>
    <b>1)</b> They use the same genotyping platform and chip
    <b>2)</b> They use the same genome build, SNP IDs, allele coding, and X coding
    <b>3)</b> They come from the same/similar geographic populations
</pre>

If it is appropriate to merge these samples, you can do so using plink:

In [ ]:
# E.g. merge the cohorts 1 and 2 from the same study into a single binary data set
plink --noweb --bfile cohort1 --bmerge cohort2.bed corhort2.bim cohort2.fam --make-bed --out data_merge

<h2>Recode alleles from A/B to nucleotides</h2>

Occasionally, alleles in a plink file are listed as A/B instead of the actual alleles.
If a SNP is associated to a phenotype, this will make it difficult to determine
which genotype is linked to the phenotype.

We can use plink --update-alleles and the file HumanOmni1-Quad_v1-0_B.update_alleles.txt
from http://www.well.ox.ac.uk/~wrayner/strand/ABtoTOPstrand.html to convert Illumina A/B genotype calls to the actual nucleotides used on the array:

In [ ]:
# E.g. recode alleles in the a data set from A/B to A/T/C/G
plink --noweb --bfile data --update-alleles HumanOmni1-Quad_v1-0_B.update_alleles.txt --make-bed --out data_recode_gen

<h2>Recode phenotypes</h2>

When you run plink (with no flags) on your data, it will output a message to the terminal that shows how it
is parsing the data. This includes the number of cases and controls it is reading from the .fam file. If both of these counts are 0, you must recode the phenotype data in the .fam file. You can extract phenotype data from a separate phenotype file included in the dbGaP download package using the script recode_phenotype.py and generate new plink files with plink:

In [ ]:
## E.g. recode phenotypes from 0 to 1/2 (control/case):

# Make an intermediate file containing phenotype data with the script recode_phenotype.py
python recode_phenotype.py data.fam phen_data.txt data_recode_phen

# Get phenotypes using plink --pheno
plink --noweb --bfile data --pheno data_recode_phen.txt --make-bed --out data_recoded1

# Explicitly code phenotypes as binary (1=control, 2=case), or they'll be read as quantitative
plink --noweb --bfile data_recoded1 --make-pheno data_recode_phen 2 --make-bed --out data_recoded2

<h2>Convert genome build with liftOver</h2>

If you intend to perform imputation or meta-analysis, you will need to make sure that all
data sets used have the same genome build (and therefore the same SNP positions and IDs).

To convert SNP positions and IDs from one build to another, you can use the liftOver tool
and appropriate chain file (http://genome.sph.umich.edu/wiki/LiftOver).

This requires some reformatting of the data as intermediate steps.

In [ ]:
## 1. Format binary plink data for input into liftOver

# a. Generate non-binary files
plink --noweb --bfile data --recode --out nb_data

# b. Add 'chr' to chromosome column of .map file, add a new column with 1-based position-1,
# and rearrange the columns into a .bed file of the correct format for liftOver
awk '{$0="chr"$0}{print $1,($4-1),$4,$2}' nb_data.map > liftOver_input.bed

# c. If necessary, change chr23 to chrX (or all X SNPs will be filtered out by liftOver)
awk '{if ($1=="chr23")sub($1,"chrX"); print $0}' liftOver_input.bed > liftOver_inputX.bed


## 2. Run liftOver on the original binary file (e.g. hg18 --> hg19)
liftOver liftOver_inputX.bed hg18ToHg19.over.chain.gz data_hg19.bed data_unlifted.txt


## 3. Convert lifted .bed file back to .map file format
awk '{print substr($1,4), $4, "0", $2}' data_hg19.bed > data_hg19.map


## 4. Use plink to exclude all unlifted hg18 SNPs from the .ped file generated in step 1a

# a. Filter out unlifted SNPs
plink --noweb --file data --exclude data_unlifted.txt --recode --out data_hg19_filtered

# b. Rename the filtered .ped file to match the hg19 .map file generated in step 3:
mv data_hg19_filtered.ped data_hg19.ped

# c. Recode as binary files
plink --noweb --file data_hg19 --make-bed --out data_hg19

<h2>Strand conversion</h2>

If you are planning to impute your data or combine it with other data sets for meta-analysis, you will need to make sure that all markers in all SNP data sets and the imputatiom reference files use allele from the same DNA strand. In both Illumina and Affy arrays, the probes target a mix of the + and - strands. In Illumina arrays, strand is coded as 'TOP' or 'BOTTOM', which has no correlation to + or -.

I have found it easiset to convert all SNPs to the + strand, since that is what our imputation reference files use. Will Rayner has put together reference files and a script to perform conversion to the + strand - that is, pulling the + strand allele for each marker. These can be downloaded here: http://www.well.ox.ac.uk/~wrayner/strand/.

In [ ]:
## Use update_build.sh to generate a new file stem with all SNPs converted to + strand
# Make sure plink dir is in path so that update_build.sh can find it

# for Illumina hg19:
update_build.sh data HumanOmni1-Quad_v1-0_B-b37.strand data_pstrand

# for Affy hg19:
update_build.sh data GenomeWide_6-b37.58-v2.strand data_pstrand

<h2>Convert Affy SNP IDs to rsIDs</h2>

This is another step that is necessary for the imputation process - only SNPs with recognized IDs will be used during imputation, and the IDs in the reference files are rsIDs.

This can get complicated because multiple Affy SNP IDs may correspond to the same rsID. Sometimes this happens because two different formats of Affy IDs are included in the array annotation file.

I have written a script that takes this into account and records duplicates in a .txt file for inspection and removal. It requires the appropriate Affy array annotation file (available on the Affy website). In this example, I use the file downloaded from http://www.affymetrix.com/estore/catalog/131533/AFFY/Genome-Wide+Human+SNP+Array+6.0#1_3

In [ ]:
## Recode Affy SNP IDs to rsIDs
# a. Generate list of rsIDs for each Affy SNP ID with recode_IDs.py and the Affy array annotation file
python custom_scripts/recode_IDs.py data.bim GenomeWideSNP_6.na35.annot.csv data_AffyID_rsID data_remove

#b. Generate new binary files with all Affy SNP IDs converted to rsIDs, and remove duplicate SNPs
plink --noweb --bfile data --update-map data_AffyID_rsID.txt --update-name --exclude data_remove.txt --make-bed --out data_rsIDs 

<h2>Re-check allele concordance with imputation ref files</h2>

Re-run check_genome_build_and_strange_alignment.pl to makes sure that your data matches up well with the imputation reference files of the corresponding build. Whether or not you plan to do impuatation, this is a good sanity check.

All metrics should be ~95% or higher if all of the above problems have been resolved.

In [ ]:
# Check allele concordance between the pre-QCed .bim file and the imputation reference files
perl /custom_scripts/check_genome_build_and_strange_alignment.pl qced_data.bim /reference_files/chrX.hg19.legend qced_data_check_hg19_X.txt

<h2>Running XWAS QC1</h2>

Once you are confident that you've done all necessary pre-processing, you can run your processed data through the initial XWAS QC script, which performs filtering on SNPs based on X-specific features (see the XWAS manual for details). The output of this step can be used as input for imputation, or XWAS tests.

In [ ]:
# In a new subdirectory, make a new parameter file with correct file stem, 
# copy the pre-processed .bed/bim/fam files into it, and run the XWAS QC script
# (see XWAS manual for details)
xwas/bin/run_QC.sh data_params_qc.txt